In [1]:
import findspark
findspark.init()
findspark.find()

'C:\\spark\\spark3'

In [2]:
from datetime import datetime
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, date_format

In [3]:
from pyspark.sql.functions import when
from pyspark.sql.functions import col,lit

In [4]:
from pyspark.sql.types import StructType,StructField,StringType,LongType,IntegerType

In [5]:
 spark = SparkSession \
        .builder \
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
        .getOrCreate()

In [6]:
output_path = r"C:\Users\goelp\OneDrive\Desktop\Output_files"

In [7]:
Schem = StructType(
[
  
  StructField("Account_No",LongType(),False),
  StructField("Age",IntegerType(),False),
  StructField("Gender",StringType(),False),
  StructField("Education",StringType(),False),
  StructField("Marital_Status",StringType(),False),
  StructField("Acc_Type",StringType(),False),
  StructField("Months_On_Bank",IntegerType(),False),
  StructField("Credit_Card",StringType(),False),
  StructField("Credit_Score",IntegerType(),False),
  StructField("Estimated_Income",StringType(),False),
  StructField("Cust_id",IntegerType(),False),
   
    
]

)

In [22]:
Bank_df = (
      spark.read
            .option("badRecordsPath","/tmp/badRecordsPath") \
            .csv(os.path.join(r"C:\Users\goelp\OneDrive\Desktop\archive/"+"Bank Customer Details.csv"),header = True,schema = Schem)
    )

In [23]:
Bank_df.printSchema()

root
 |-- Account_No: long (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Education: string (nullable = true)
 |-- Marital_Status: string (nullable = true)
 |-- Acc_Type: string (nullable = true)
 |-- Months_On_Bank: integer (nullable = true)
 |-- Credit_Card: string (nullable = true)
 |-- Credit_Score: integer (nullable = true)
 |-- Estimated_Income: string (nullable = true)
 |-- Cust_id: integer (nullable = true)



In [24]:
Bank_df.show()

+----------+---+------+-------------+--------------+--------+--------------+-----------+------------+----------------+-------+
|Account_No|Age|Gender|    Education|Marital_Status|Acc_Type|Months_On_Bank|Credit_Card|Credit_Score|Estimated_Income|Cust_id|
+----------+---+------+-------------+--------------+--------+--------------+-----------+------------+----------------+-------+
|4998365300| 45|  Male|  High School|       Married| Savings|            42|        Yes|         619|       60K - 80K|      1|
|4998365301| 49|Female|     Graduate|        Single| Savings|            41|         No|         608|   Less than 40K|      2|
|4998365302| 51|  Male|     Graduate|       Married| Savings|            42|        Yes|         502|      80K - 120K|      3|
|4998365303| 40|Female|  High School|          null| Savings|            39|         No|         699|   Less than 40K|      4|
|4998365304| 40|  Male|   Uneducated|       Married| Savings|            43|        Yes|         850|       60K

In [13]:
Customer = Bank_df.select("Cust_id",
                      "Gender",
                      "Education",
                      "Marital_Status",
                       "Age",
                       "Account_No"
                    ).drop_duplicates()

In [14]:
Customer.show()

+-------+------+-------------+--------------+---+----------+
|Cust_id|Gender|    Education|Marital_Status|Age|Account_No|
+-------+------+-------------+--------------+---+----------+
|     91|Female|  High School|        Single| 51|4998365390|
|    331|Female|     Graduate|        Single| 43|4998365630|
|    465|  Male|     Graduate|        Single| 50|4998365764|
|    635|  Male|   Uneducated|       Married| 65|4998365934|
|    775|  Male|      College|        Single| 46|4998366074|
|    838|  Male|      College|       Married| 56|4998366137|
|    909|Female|  High School|        Single| 46|4998366208|
|   1080|Female|     Graduate|       Married| 45|4998366379|
|   1330|  Male|     Graduate|       Married| 33|4998366629|
|   1447|  Male|   Uneducated|      Divorced| 54|4998366746|
|   1532|  Male|      Unknown|       Married| 39|4998366831|
|   1582|Female|  High School|        Single| 33|4998366881|
|   2275|Female|     Graduate|       Married| 61|4998367574|
|   2370|  Male|      Un

In [15]:
Account = Bank_df.select("Account_No",
                    "Acc_Type",
                    "Months_On_Bank",
                    "Credit_Score",
                    "Credit_Card",
                    "Estimated_Income"
                    ).drop_duplicates()

In [16]:
Customer.write.mode('overwrite').parquet(output_path+'Customer/')

In [25]:
Account.write.mode('overwrite').parquet(output_path+'Account/')

In [27]:
df = spark.read.csv(r"C:\Users\goelp\OneDrive\Desktop\archive\bank.csv",header = True,inferSchema = True)

In [28]:
df.show(5)

+----------+---------+--------------------+------+----------+------------+------------+------------+-------+
|Account_No|     DATE|       TRANS_DETAILS|CHQ_NO|VALUE_DATE|WITHDRAW_AMT| DEPOSIT_AMT| BALANCE_AMT|Tras_id|
+----------+---------+--------------------+------+----------+------------+------------+------------+-------+
|4998365300|29-Jun-17|TRF FROM  Indiafo...|  null| 29-Jun-17|        null|10,00,000.00|10,00,000.00|      1|
|4998365301|05-Jul-17|TRF FROM  Indiafo...|  null| 05-Jul-17|        null|10,00,000.00|20,00,000.00|      2|
|4998365302|18-Jul-17|FDRL/INTERNAL FUN...|  null| 18-Jul-17|        null| 5,00,000.00|25,00,000.00|      3|
|4998365303|01-Aug-17|TRF FRM  Indiafor...|  null| 01-Aug-17|        null|30,00,000.00|55,00,000.00|      4|
|4998365304|16-Aug-17|FDRL/INTERNAL FUN...|  null| 16-Aug-17|        null| 5,00,000.00|60,00,000.00|      5|
+----------+---------+--------------------+------+----------+------------+------------+------------+-------+
only showing top 5 

In [29]:
df1 = df.na.fill("")

In [30]:
df1.show()

+----------+---------+--------------------+------+----------+------------+------------+------------+-------+
|Account_No|     DATE|       TRANS_DETAILS|CHQ_NO|VALUE_DATE|WITHDRAW_AMT| DEPOSIT_AMT| BALANCE_AMT|Tras_id|
+----------+---------+--------------------+------+----------+------------+------------+------------+-------+
|4998365300|29-Jun-17|TRF FROM  Indiafo...|  null| 29-Jun-17|            |10,00,000.00|10,00,000.00|      1|
|4998365301|05-Jul-17|TRF FROM  Indiafo...|  null| 05-Jul-17|            |10,00,000.00|20,00,000.00|      2|
|4998365302|18-Jul-17|FDRL/INTERNAL FUN...|  null| 18-Jul-17|            | 5,00,000.00|25,00,000.00|      3|
|4998365303|01-Aug-17|TRF FRM  Indiafor...|  null| 01-Aug-17|            |30,00,000.00|55,00,000.00|      4|
|4998365304|16-Aug-17|FDRL/INTERNAL FUN...|  null| 16-Aug-17|            | 5,00,000.00|60,00,000.00|      5|
|4998365305|16-Aug-17|FDRL/INTERNAL FUN...|  null| 16-Aug-17|            | 5,00,000.00|65,00,000.00|      6|
|4998365306|16-Aug-

In [31]:
Bank_log_details = df1.select("Account_No",
                              "DATE",
                              "TRANS_DETAILS",
                              "CHQ_NO",
                              "VALUE_DATE",
                              "WITHDRAW_AMT",
                              "DEPOSIT_AMT",
                              "BALANCE_AMT",
                              "Tras_id").drop_duplicates()

In [32]:
Bank_log_details = Bank_log_details.withColumn("Action", \
   when((Bank_log_details.WITHDRAW_AMT !=""), lit("Withdraw"))\
     .otherwise(lit("Deposit")))

In [33]:
Bank_log_details.show()

+----------+---------+--------------------+------+----------+------------+------------+----------------+-------+--------+
|Account_No|     DATE|       TRANS_DETAILS|CHQ_NO|VALUE_DATE|WITHDRAW_AMT| DEPOSIT_AMT|     BALANCE_AMT|Tras_id|  Action|
+----------+---------+--------------------+------+----------+------------+------------+----------------+-------+--------+
|4998365327|16-Aug-17|INDO GIBL Indiafo...|  null| 16-Aug-17|   78,100.00|            |    62,01,467.00|     28|Withdraw|
|4998365561|07-Feb-18|INDO GIBL Indiafo...|  null| 07-Feb-18|   61,500.00|            |    14,35,695.00|    262|Withdraw|
|4998366033|29-Sep-18|Indfor INCOME IND...|  null| 29-Sep-18|    1,920.00|            |    10,28,562.00|    734|Withdraw|
|4998366937|01-Mar-18|FDRL/INTERNAL FUN...|  null| 01-Mar-18|            | 3,00,000.00|    11,04,941.41|   1638| Deposit|
|4998366986|09-Apr-18|FDRL/INTERNAL FUN...|  null| 09-Apr-18|            | 3,00,000.00|     8,80,613.78|   1687| Deposit|
|4998367070|05-Jun-18|  

In [34]:
Bank_log_details.write.mode('overwrite').parquet(os.path.join(output_path+'Bank_log/'))

In [35]:
Customer_details = spark.read.parquet(os.path.join(output_path+'Customer/'))

In [36]:
Account_details = spark.read.parquet(os.path.join(output_path+'Account/'))

In [37]:
from pyspark.sql.functions import monotonically_increasing_id

In [38]:
Bank_trans = Bank_log_details.join(Customer_details, Bank_log_details.Account_No == Customer_details.Account_No, how='inner')\
                        .select(monotonically_increasing_id().alias("Trans_id"),\
                        col("Cust_id"),\
                        df1.Account_No,\
                        "Action",\
                        "DATE",
                        "VALUE_DATE"\
                        )

In [39]:
Bank_trans = Bank_trans.join(Account_details, Bank_trans.Account_No == Account_details.Account_No, how='inner')\
                        .select(monotonically_increasing_id().alias("Trans_id"),\
                        col("Cust_id"),\
                        Bank_trans.Account_No,\
                        col("Action"),\
                        col("DATE"),
                        col("VALUE_DATE"),\
                        "Acc_type"
                        )

In [40]:
Bank_trans.show()

+--------+-------+----------+--------+---------+----------+--------+
|Trans_id|Cust_id|Account_No|  Action|     DATE|VALUE_DATE|Acc_type|
+--------+-------+----------+--------+---------+----------+--------+
|       0|     28|4998365327|Withdraw|16-Aug-17| 16-Aug-17| Savings|
|       1|    262|4998365561|Withdraw|07-Feb-18| 07-Feb-18| Savings|
|       2|    734|4998366033|Withdraw|29-Sep-18| 29-Sep-18| Savings|
|       3|   1638|4998366937| Deposit|01-Mar-18| 01-Mar-18| Savings|
|       4|   1687|4998366986| Deposit|09-Apr-18| 09-Apr-18| Savings|
|       5|   1771|4998367070|Withdraw|05-Jun-18| 05-Jun-18| Savings|
|       6|   1943|4998367242| Deposit|02-Nov-18| 02-Nov-18| Savings|
|       7|   2250|4998367549| Deposit|30-Jul-16| 30-Jul-16| Savings|
|       8|   2377|4998367676| Deposit|26-Sep-16| 26-Sep-16| Savings|
|       9|   2392|4998367691| Deposit|29-Sep-16| 29-Sep-16| Savings|
|      10|   3107|4998368406| Deposit|24-Aug-16| 24-Aug-16| Savings|
|      11|   3251|4998368550| Depo

In [41]:
Bank_trans.write.mode('overwrite').parquet(output_path+'Bank_trans/')

In [42]:
Customer.write.mode('overwrite').parquet(output_path+'Customer/')